In [1]:
# Import necessary libraries
import random
import json
import pickle
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.utils.class_weight import compute_class_weight

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
# Load and preprocess data
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Process intents and patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and clean words
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

# Save words and classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [3]:
# Define a function to get synonyms
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() != word and lemma.name() not in synonyms:
                synonyms.append(lemma.name())
    return synonyms[:2]  # Return up to 2 synonyms

# Augment training data
augmented_documents = documents.copy()
for doc, tag in documents:
    augmented_pattern = []
    for word in doc:
        synonyms = get_synonyms(word)
        if synonyms:
            # Add a pattern with one word replaced by its synonym
            for syn in synonyms:
                new_pattern = doc.copy()
                new_pattern[new_pattern.index(word)] = syn
                augmented_documents.append((new_pattern, tag))

documents = augmented_documents

In [4]:
# Prepare training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    # Create bag of words
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append(bag + output_row)

# Shuffle and convert to numpy array
random.shuffle(training)
training = np.array(training)

# Split features and labels
X = training[:, :len(words)]
y = training[:, len(words):]

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Build the model with improved architecture
model = tf.keras.Sequential([
    # Input layer with more units and batch normalization
    tf.keras.layers.Dense(512, input_shape=(len(words),), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    # Hidden layers with decreasing units
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    # Output layer
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

# Compile with modified parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
# Modified early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    patience=30,            # Increased patience
    restore_best_weights=True,
    min_delta=0.001        # Minimum change to qualify as an improvement
)

# Add learning rate reduction
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=10,
    min_lr=0.0001
)

# Get the true labels (convert one-hot encoded y_train to class indices)
y_train_labels = np.argmax(y_train, axis=1)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

# Convert to dictionary
class_weight_dict = dict(zip(np.unique(y_train_labels), class_weights))

In [7]:
# Train with modified parameters
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=500,            
    batch_size=16,        
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict  # Use the computed class weights
)

Epoch 1/500
340/340 [==============================] - 4s 6ms/step - loss: 3.4786 - accuracy: 0.3901 - val_loss: 3.0626 - val_accuracy: 0.7351 - lr: 0.0010
Epoch 2/500
340/340 [==============================] - 1s 4ms/step - loss: 0.8942 - accuracy: 0.8633 - val_loss: 0.3264 - val_accuracy: 0.9279 - lr: 0.0010
Epoch 3/500
340/340 [==============================] - 2s 5ms/step - loss: 0.4037 - accuracy: 0.9268 - val_loss: 0.2144 - val_accuracy: 0.9352 - lr: 0.0010
Epoch 4/500
340/340 [==============================] - 2s 5ms/step - loss: 0.3139 - accuracy: 0.9280 - val_loss: 0.2163 - val_accuracy: 0.9338 - lr: 0.0010
Epoch 5/500
340/340 [==============================] - 2s 5ms/step - loss: 0.2721 - accuracy: 0.9367 - val_loss: 0.2070 - val_accuracy: 0.9367 - lr: 0.0010
Epoch 6/500
340/340 [==============================] - 1s 4ms/step - loss: 0.2703 - accuracy: 0.9374 - val_loss: 0.2110 - val_accuracy: 0.9433 - lr: 0.0010
Epoch 7/500
340/340 [==============================] - 1s 4ms/st

In [8]:
# Print final metrics
final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
print(f'Final Training Accuracy: {final_train_accuracy:.4f}')
print(f'Final Validation Accuracy: {final_val_accuracy:.4f}')

Final Training Accuracy: 0.9590
Final Validation Accuracy: 0.9360


In [9]:
# Save the model
model.save('chatbot_model.h5')
print('Training completed and model saved!')

Training completed and model saved!
